<a href="https://colab.research.google.com/github/Downforcedemon/AI/blob/main/Train_Daneel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install necessary libraries
!pip install transformers accelerate peft datasets torch torch_xla -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.8/374.8 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 55.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.9/274.9 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.0/129.0 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.1/231.1 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.1/344.1 kB 25.9 MB/s eta 0:00:00


In [4]:
# import required libraries
import torch
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model
import torch_xla.core.xla_model as xm
from datasets import load_dataset
import os
import json

🚀 Step 2: Load and Prepare the Daneel Dialogue Dataset
📌 What We Will Do in This Step

    Load daneel_dialogue_cleaned.json into memory.
    Flatten the structured format (convert the dataset into a simple input-output format).
    Prepare data for tokenization by combining dialogues into structured training samples.

In [3]:
# Define dataset path
dataset_path = "/content/daneel_dialogue_cleaned.json"

# Load the cleaned dataset
with open(dataset_path, "r", encoding="utf-8") as f:
    raw_data = json.load(f)

# Flatten the dataset: Extract Q&A pairs
training_data = []
for entry in raw_data:
    context = entry["context"]
    for dialogue in entry["dialogue"]:
        input_text = dialogue["input"]
        output_text = dialogue["output"]

        # Format training example as a conversational turn
        formatted_sample = {
            "input": f"User: {input_text}\nDaneel: ",
            "output": output_text
        }
        training_data.append(formatted_sample)

# Print dataset sample
print(f"Total training samples: {len(training_data)}")
print("Sample entry:\n", training_data[0])


Total training samples: 11012
Sample entry:
 {'input': 'User: v1.0 formatting and some spellchecking isaac asimov the robots of dawn doubleday company, inc\nDaneel: ', 'output': 'garden city, new york by nightfall, inc'}


🚀 Step 3: Load the DeepSeek Model and Tokenizer
📌 What We Will Do in This Step

    Load DeepSeek Model (deepseek-ai/deepseek-coder-7b or deepseek-coder-8b).
    Load DeepSeek Tokenizer (to tokenize our training data).
    Ensure Model Runs on TPU (v2-8) using torch_xla.
    Verify model loading by generating a sample output.

In [11]:
!pip install -U bitsandbytes  # Ensure bitsandbytes is installed and up-to-date

import bitsandbytes as bnb # Now import bitsandbytes




In [13]:


# Define the DeepSeek model checkpoint
model_name = "deepseek-ai/deepseek-coder-7b"

# Load DeepSeek Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load DeepSeek Model with 8-bit precision
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_8bit=True,  # Load in 8-bit precision to reduce memory usage
    device_map="auto"  # Automatically distribute model layers
)

# Verify model load by generating a sample response
sample_input = "User: What are the Three Laws of Robotics?\nDaneel:"
input_ids = tokenizer(sample_input, return_tensors="pt").input_ids.to(model.device)  # Move inputs to the same device
output_ids = model.generate(input_ids, max_length=100)
output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

print("Sample Model Output:\n", output_text)


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


ImportError: Using `bitsandbytes` 8-bit quantization requires the latest version of bitsandbytes: `pip install -U bitsandbytes`